In [3]:
from dotenv import load_dotenv
from openai import AsyncOpenAI
from agents import Agent, Runner, trace, function_tool, OpenAIChatCompletionsModel, input_guardrail, GuardrailFunctionOutput, InputGuardrailTripwireTriggered
from typing import Dict, Optional, List
from pydantic import BaseModel
import os
from twilio.rest import Client  # WhatsApp messaging via Twilio
load_dotenv(override=True)


True

In [6]:

openai_api_key = os.getenv('OPENAI_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')
deepseek_api_key = os.getenv('DEEPSEEK_API_KEY')
groq_api_key = os.getenv('GROQ_API_KEY')

# Check if essential API keys are set
if not openai_api_key:
    raise ValueError("OPENAI_API_KEY environment variable not set. Please add it to your .env file.")

# Initialize the OpenAI client
openai_client = AsyncOpenAI(api_key=openai_api_key)

print("API keys and clients initialized.")

API keys and clients initialized.


In [7]:
instructions_professional = "You are a professional social media content creator working for 'Elite Digital', a marketing agency specializing in B2B tech clients. You craft precise, informative, and engaging social media posts for platforms like LinkedIn."

instructions_humorous = "You are a witty and engaging social media content creator working for 'Elite Digital'. You write humorous, trending, and shareable social media posts for platforms like X (formerly Twitter) and Instagram."

instructions_concise = "You are a busy social media content creator working for 'Elite Digital'. You write concise, to-the-point social media posts ideal for short-form content like Instagram Stories or quick updates."

In [8]:
GEMINI_BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"
DEEPSEEK_BASE_URL = "https://api.deepseek.com/v1"
GROQ_BASE_URL = "https://api.groq.com/openai/v1"

In [9]:
# Initialize AsyncOpenAI clients for different API providers
deepseek_client = AsyncOpenAI(base_url=DEEPSEEK_BASE_URL, api_key=deepseek_api_key)
gemini_client = AsyncOpenAI(base_url=GEMINI_BASE_URL, api_key=google_api_key)
groq_client = AsyncOpenAI(base_url=GROQ_BASE_URL, api_key=groq_api_key)

# Define models for each client
deepseek_model = OpenAIChatCompletionsModel(model="deepseek-chat", openai_client=deepseek_client)
gemini_model = OpenAIChatCompletionsModel(model="gemini-2.0-flash", openai_client=gemini_client)
llama3_8b_model = OpenAIChatCompletionsModel(model="llama3-8b-8192", openai_client=groq_client)

In [10]:
# Create agents for generating social media posts with different tones
post_agent_professional = Agent(name="Professional Post Agent", instructions=instructions_professional, model=deepseek_model)
post_agent_humorous = Agent(name="Humorous Post Agent", instructions=instructions_humorous, model=gemini_model)
post_agent_concise = Agent(name="Concise Post Agent", instructions=instructions_concise, model=llama3_8b_model)

In [11]:
# Define a common description for post generation tools
description_post_gen = "Generate a social media post for a given topic."

# Convert social media agents into tools
tool_professional_post = post_agent_professional.as_tool(tool_name="professional_post_generator", tool_description=description_post_gen)
tool_humorous_post = post_agent_humorous.as_tool(tool_name="humorous_post_generator", tool_description=description_post_gen)
tool_concise_post = post_agent_concise.as_tool(tool_name="concise_post_generator", tool_description=description_post_gen)

In [12]:
class WhatsAppMessage(BaseModel):
    content: str
    to_number: str  # WhatsApp number with country code
    message_type: Optional[str] = "social_media_post"

@function_tool
def send_whatsapp_message(to_number: str, message_content: str) -> Dict[str, str]:
    """
    Sends a message to a WhatsApp number using the Twilio API.
    Args:
        to_number (str): The recipient's WhatsApp number (e.g., "+1234567890").
                         Must include the country code and be formatted correctly.
        message_content (str): The content of the message to send.
    Returns:
        Dict[str, str]: A dictionary indicating the status of the message sending.
    """
    account_sid = os.environ.get('WHATSAPP_ACCOUNT_SID')
    auth_token = os.environ.get('WHATSAPP_AUTH_TOKEN')
    from_number = os.environ.get('WHATSAPP_FROM_NUMBER')

    if not all([account_sid, auth_token, from_number]):
        return {"status": "error", "message": "WhatsApp API credentials (ACCOUNT_SID, AUTH_TOKEN, FROM_NUMBER) not found in environment variables. Please check your .env file."}

    try:
        client = Client(account_sid, auth_token)
        message = client.messages.create(
            from_=f'whatsapp:{from_number}',
            body=message_content,
            to=f'whatsapp:{to_number}'
        )
        return {"status": "success", "sid": message.sid, "message": f"Message sent to WhatsApp successfully. SID: {message.sid}"}
    except Exception as e:
        return {"status": "error", "message": f"Failed to send WhatsApp message: {e}"}


In [13]:
whatsapp_formatter_instructions = """You are a WhatsApp message formatter for 'Elite Digital'. You receive a social media post in text format and need to format it appropriately for WhatsApp messaging. 

WhatsApp formatting guidelines:
- Use emojis strategically to make the message engaging
- Break long text into readable paragraphs
- Use *bold* for emphasis where appropriate
- Keep hashtags relevant and not overwhelming
- Ensure the message is mobile-friendly and easy to read
- Adapt the tone to be conversational while maintaining professionalism

Focus on making the message clear, engaging, and suitable for WhatsApp's messaging format."""

whatsapp_formatter = Agent(
    name="WhatsApp Message Formatter", 
    instructions=whatsapp_formatter_instructions,
    model="gpt-4o-mini"
)

# Convert formatter to tool
whatsapp_format_tool = whatsapp_formatter.as_tool(
    tool_name="format_for_whatsapp",
    tool_description="Format a raw social media post text for WhatsApp messaging with appropriate formatting and emojis."
)

In [14]:
whatsapp_tools = [whatsapp_format_tool, send_whatsapp_message]

whatsapp_dispatcher_instructions = """You are a WhatsApp message dispatcher for 'Elite Digital'. You receive a final social media post and a WhatsApp number. Your task is to:

1. First use the 'format_for_whatsapp' tool to format the post appropriately for WhatsApp
2. Then use the 'send_whatsapp_message' tool to send it to the specified WhatsApp number

The WhatsApp number should be provided in the input. If not provided, ask for it or use a default test number."""

whatsapp_dispatcher_agent = Agent(
    name="WhatsApp Dispatcher",
    instructions=whatsapp_dispatcher_instructions,
    tools=whatsapp_tools,
    model="gpt-4o-mini",
    handoff_description="Format a social media post for WhatsApp and send it."
)

In [15]:
post_generation_tools = [tool_professional_post, tool_humorous_post, tool_concise_post]
handoff_agents = [whatsapp_dispatcher_agent]


In [16]:
social_media_manager_instructions = """
You are a Social Media Manager for 'Elite Digital'. Your task is to create compelling social media posts using 3 specialized agents:
- A professional tone agent
- A humorous tone agent  
- A concise tone agent

You must:
1. Use each of the 3 social media tools exactly once.
2. Compare the 3 outputs.
3. Select the single best one using your judgment of which is most effective and suitable for the given platform.
4. Handoff the selected post to the 'WhatsApp Dispatcher' agent for formatting and sending.

The WhatsApp number will be provided in the input. Extract it and include it when handing off to the dispatcher.

Do not loop or retry. Only call each tool once.
"""

social_media_manager = Agent(
    name="Social Media Manager",
    instructions=social_media_manager_instructions.strip(),
    tools=post_generation_tools,
    handoffs=handoff_agents,
    model="gpt-4o-mini"
)

# Test the system
message_topic = "Generate a social media post about the benefits of AI in cybersecurity for LinkedIn. Send to WhatsApp number +4915222350056."

print("Running Social Media Manager with WhatsApp integration...")
with trace("Automated Social Media Post to WhatsApp") as t:
    result = await Runner.run(social_media_manager, message_topic)
print(result.final_output)

Running Social Media Manager with WhatsApp integration...
The social media post about the benefits of AI in cybersecurity has been successfully sent to the WhatsApp number +4915222350056! 🎉

### Message Details:
- **Status**: Sent successfully
- **Message SID**: SM1b24c668d45da2d88ef6f612bda14a9a

If you need any further assistance or have another task, feel free to let me know!


In [ ]:
class KeywordCheckOutput(BaseModel):
    contains_sensitive_keyword: bool
    found_keywords: List[str]

guardrail_keyword_agent = Agent(
    name="Keyword Check Guardrail",
    instructions="Check if the user's request contains any sensitive or restricted keywords related to controversial topics, competitive names, or client confidential information. Respond with a boolean indicating if such keywords were found and a list of the keywords if any.",
    output_type=KeywordCheckOutput,
    model="gpt-4o-mini"
)

@input_guardrail
async def guardrail_against_sensitive_keywords(ctx, agent, message):
    """
    A guardrail to check if the user's input message contains any sensitive keywords.
    Trips the wire if sensitive keywords are detected.
    """
    result = await Runner.run(guardrail_keyword_agent, message, context=ctx.context)
    contains_sensitive = result.final_output.contains_sensitive_keyword
    return GuardrailFunctionOutput(
        output_info={"found_keywords": result.final_output.found_keywords},
        tripwire_triggered=contains_sensitive
    )

In [16]:
careful_social_media_manager = Agent(
    name="Careful Social Media Manager",
    instructions=social_media_manager_instructions,
    tools=post_generation_tools,
    handoffs=handoff_agents,
    model="gpt-4o-mini",
    input_guardrails=[guardrail_against_sensitive_keywords]
)

# Test message expected to trip the guardrail
message_guarded_fail = "Create a social media post about why 'CompetitorX' is inferior for B2B cybersecurity. Send to WhatsApp +4915222350056."

print("\n--- Running with Guardrail (should be tripped) ---")
with trace("Protected Social Media Post to WhatsApp - Fail"):
    try:
        result_guarded_fail = await Runner.run(careful_social_media_manager, message_guarded_fail)
        print(f"Guarded Result (Fail): {result_guarded_fail}")
    except InputGuardrailTripwireTriggered as e:
        print(f"❌ Guardrail Tripped! Input blocked for: '{message_guarded_fail}'")
        print(f"🔒 Guardrail Name: {e.guardrail_result.guardrail.get_name()}")
        print("📋 Full Guardrail Result:", e.guardrail_result)

# Test message expected to pass the guardrail
message_guarded_success = "Generate a social media post about the importance of data privacy in cloud computing. Send to WhatsApp +4915222350056."

print("\n--- Running with Guardrail (should succeed) ---")
with trace("Protected Social Media Post to WhatsApp - Success"):
    try:
        result_guarded_success = await Runner.run(careful_social_media_manager, message_guarded_success)
        print(f"✅ Guarded Result (Success): {result_guarded_success}")
    except InputGuardrailTripwireTriggered as e:
        print(f"❌ Unexpected Guardrail Trip for: '{message_guarded_success}'")
        print(f"🔒 Guardrail Name: {e.guardrail_result.guardrail.get_name()}")
        print("📋 Full Guardrail Result:", e.guardrail_result)